# Flight Prediction Web Scraping

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from selenium import webdriver 
from datetime import date 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from dateutil.relativedelta import relativedelta
import csv
import linecache
import sys

In [2]:
def PrintException():
    exc_type,exc_obj,tb = sys.exc_info()
    f= tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno,f.f_globals)
    print ('Exception in ({},Line {} "{}"):{}'.format(filename, lineno,line.strip, exc_obj))

In [3]:
def scrap (origin, destin,trDate):
    global none
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|"
    try:
        driver = webdriver.Chrome(r"C:\driver\chromedriver.exe")
        print("Requested Url: "+baseDataUrl)
        
        driver.get(baseDataUrl)
        print("WebPage found...")
        
        element_xpath = '//*[@id="left-side--wrapper"]/div[2]'
        
        print("Scrolling document upto bottom...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})
        # Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})
        # Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 

In [5]:
dates=(pd.date_range(start='05/05/2022', end='05/11/2022'))
New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year  
    New_date.append(date)
print(New_date)

['05/05/2022', '06/05/2022', '07/05/2022', '08/05/2022', '09/05/2022', '10/05/2022', '11/05/2022']


In [6]:
#Lists to store scraped data
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    df.to_csv('Flight.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [7]:
list_of_dest=['GOI','AMD','TIR','BLR','BOM','DEL']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")

--------------
origin = GOI
destin = GOI

origin = GOI
destin = AMD

Requested Url: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-05/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 5
Time Count in Scraping  = 10
City Count in Scraping  = 12
spanFlightCost Count in Scraping  = 5
pFlightStops Count in Scraping  = 5
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IndiGo,05/05/2022,Goa,Ahmedabad,01:20,03:00,Non stop,"₹ 6,471"
2,SpiceJet,05/05/2022,Goa,Ahmedabad,07:50,09:30,Non stop,"₹ 6,471"
3,Go First,05/05/2022,Goa,Ahmedabad,21:00,22:30,Non stop,"₹ 6,471"
4,IndiGo,05/05/2022,Goa,Ahmedabad,00:10,07:40,1 stop via Mumbai,"₹ 6,471"
5,IndiGo,05/05/2022,Goa,Ahmedabad,20:15,21:55,Non stop,"₹ 12,771"


Lenght :  6
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-06/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 5
Time Count in Scraping  = 10
City Count in Scraping  = 11
spanFlightCost Count in Scraping  = 5
pFlightStops Count in Scraping  = 5
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IndiGo,05/05/2022,Goa,Ahmedabad,01:20,03:00,Non stop,"₹ 6,471"
2,SpiceJet,05/05/2022,Goa,Ahmedabad,07:50,09:30,Non stop,"₹ 6,471"
3,Go First,05/05/2022,Goa,Ahmedabad,21:00,22:30,Non stop,"₹ 6,471"
4,IndiGo,05/05/2022,Goa,Ahmedabad,00:10,07:40,1 stop via Mumbai,"₹ 6,471"
5,IndiGo,05/05/2022,Goa,Ahmedabad,20:15,21:55,Non stop,"₹ 12,771"
6,IndiGo,06/05/2022,Goa,Ahmedabad,01:20,03:00,Non stop,"₹ 6,471"
7,SpiceJet,06/05/2022,Goa,Ahmedabad,07:50,09:30,Non stop,"₹ 6,471"
8,IndiGo,06/05/2022,Goa,Ahmedabad,20:15,21:55,Non stop,"₹ 6,471"
9,Go First,06/05/2022,Goa,Ahmedabad,21:00,22:30,Non stop,"₹ 6,471"


Lenght :  11
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-07/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 22
Time Count in Scraping  = 44
City Count in Scraping  = 46
spanFlightCost Count in Scraping  = 22
pFlightStops Count in Scraping  = 22
Transaform Function Running.... 


...

Lenght :  4272
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-10/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 50
Time Count in Scraping  = 100
City Count in Scraping  = 101
spanFlightCost Count in Scraping  = 50
pFlightStops Count in Scraping  = 50
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IndiGo,05/05/2022,Goa,Ahmedabad,01:20,03:00,Non stop,"₹ 6,471"
2,SpiceJet,05/05/2022,Goa,Ahmedabad,07:50,09:30,Non stop,"₹ 6,471"
3,Go First,05/05/2022,Goa,Ahmedabad,21:00,22:30,Non stop,"₹ 6,471"
4,IndiGo,05/05/2022,Goa,Ahmedabad,00:10,07:40,1 stop via Mumbai,"₹ 6,471"
...,...,...,...,...,...,...,...,...
4317,Go First,10/05/2022,New Delhi,Mumbai,16:45,21:10,1 stop via Ahmedabad,"₹ 10,862"
4318,Go First,10/05/2022,New Delhi,Mumbai,15:45,18:00,Non stop,"₹ 12,519"
4319,SpiceJet,10/05/2022,New Delhi,Mumbai,19:45,22:00,Non stop,"₹ 12,779"
4320,Go First,10/05/2022,New Delhi,Mumbai,12:35,18:10,1 stop via Patna,"₹ 13,094"


Lenght :  4322
 
Requested Url: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-11/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E&ccde=IN&lang=eng&cmp=SEM|M|DF|B|Brand|B_M_Makemytrip_Search_Exact|Brand_Top_5_Exact|Expanded|
WebPage found...
Scrolling document upto bottom...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 21
Time Count in Scraping  = 42
City Count in Scraping  = 42
spanFlightCost Count in Scraping  = 21
pFlightStops Count in Scraping  = 21
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IndiGo,05/05/2022,Goa,Ahmedabad,01:20,03:00,Non stop,"₹ 6,471"
2,SpiceJet,05/05/2022,Goa,Ahmedabad,07:50,09:30,Non stop,"₹ 6,471"
3,Go First,05/05/2022,Goa,Ahmedabad,21:00,22:30,Non stop,"₹ 6,471"
4,IndiGo,05/05/2022,Goa,Ahmedabad,00:10,07:40,1 stop via Mumbai,"₹ 6,471"
...,...,...,...,...,...,...,...,...
4338,Go First,11/05/2022,New Delhi,Mumbai,14:00,16:10,Non stop,"₹ 7,319"
4339,IndiGo,11/05/2022,New Delhi,Mumbai,14:10,16:20,Non stop,"₹ 7,319"
4340,IndiGo,11/05/2022,New Delhi,Mumbai,15:25,17:35,Non stop,"₹ 7,319"
4341,Go First,11/05/2022,New Delhi,Mumbai,15:45,18:00,Non stop,"₹ 7,319"


Lenght :  4343
 
-----Skiped-----
 
origin = DEL
destin = DEL



In [11]:
#checking scraped Flight Price data
df=pd.read_csv("Flight.csv")
df

,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price (?)
0,IndiGo,05-05-2022,Goa,Ahmedabad,01:20,03:00,Non stop,"6,471"
1,SpiceJet,05-05-2022,Goa,Ahmedabad,07:50,09:30,Non stop,"6,471"
2,Go First,05-05-2022,Goa,Ahmedabad,21:00,22:30,Non stop,"6,471"
3,IndiGo,05-05-2022,Goa,Ahmedabad,00:10,07:40,1 stop via Mumbai,"6,471"
4,IndiGo,05-05-2022,Goa,Ahmedabad,20:15,21:55,Non stop,"12,771"
...,...,...,...,...,...,...,...,...
4337,Go First,11-05-2022,New Delhi,Mumbai,14:00,16:10,Non stop,"7,319"
4338,IndiGo,11-05-2022,New Delhi,Mumbai,14:10,16:20,Non stop,"7,319"
4339,IndiGo,11-05-2022,New Delhi,Mumbai,15:25,17:35,Non stop,"7,319"
4340,Go First,11-05-2022,New Delhi,Mumbai,15:45,18:00,Non stop,"7,319"


In [ ]:
#saving Flight Price data as csv file
df.to_csv("flight_price.csv")